In [ ]:
path = "/home/joakim/projects/imageAnnotationHandlers/test_train/"
model_path = "model/"
print("test")
batch_size=8

In [ ]:
import keras

In [ ]:
import vgg16
from vgg16 import Vgg16

In [ ]:
import utils; reload(utils)
from utils import plots
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
vgg = Vgg16()
model = vgg.model

In [ ]:
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'test', batch_size=batch_size*2)

In [ ]:
import bcolz
from utils import get_data
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [ ]:
val_data = get_data(path+'test')

In [ ]:
trn_data = get_data(path+'train')

In [ ]:
trn_data.shape

In [ ]:
save_array(model_path+ 'train_data.bc', trn_data)
save_array(model_path + 'valid_data.bc', val_data)

In [ ]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [ ]:
trn_data.shape

In [ ]:
import numpy as np

def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [ ]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
trn_classes.shape

In [ ]:
trn_classes[:4]

In [ ]:
trn_labels[:4]

### PS, having shuffle=True on get_batches makes a huge difference in accuracy with this sample-size

In [ ]:
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

In [ ]:
trn_features.shape

In [ ]:
save_array(model_path+ 'train_lastlayer_features.bc', trn_features)
save_array(model_path + 'valid_lastlayer_features.bc', val_features)

In [ ]:
trn_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'valid_lastlayer_features.bc')

In [ ]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

lm = Sequential([ Dense(5, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
trn_labels.shape

In [ ]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size,validation_data=(val_features, val_labels))

In [ ]:
lm.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size,validation_data=(val_features, val_labels))

In [ ]:
# We want both the classes...
preds = lm.predict_classes(val_features, batch_size=batch_size)
# ...and the probabilities of being a cat
probs = lm.predict_proba(val_features, batch_size=batch_size)[:,0]
probs[:8]

In [ ]:
preds[:8]

In [ ]:
filenames = val_batches.filenames
n_view = 4

In [ ]:
def plots_idx(idx, titles=None):
    plots([image.load_img(path + 'test/' + filenames[i]) for i in idx], titles=titles)

In [ ]:
#1. A few correct labels at random
correct = np.where(preds==val_labels[:,1])[0]
idx = permutation(correct)[:n_view]
plots_idx(idx, probs[idx])

In [ ]:
#2. A few incorrect labels at random
incorrect = np.where(preds!=val_labels[:,1])[0]
idx = permutation(incorrect)[:n_view]
plots_idx(idx, probs[idx])

In [ ]:
#3. The images we most confident were cats, and are actually cats
correct_cats = np.where((preds==0) & (preds==val_labels[:,1]))[0]
most_correct_cats = np.argsort(probs[correct_cats])[::-1][:n_view]
plots_idx(correct_cats[most_correct_cats], probs[correct_cats][most_correct_cats])

In [ ]:
# as above, but dogs
correct_dogs = np.where((preds==1) & (preds==val_labels[:,1]))[0]
most_correct_dogs = np.argsort(probs[correct_dogs])[:n_view]
plots_idx(correct_dogs[most_correct_dogs], 1-probs[correct_dogs][most_correct_dogs])

In [ ]:
#3. The images we were most confident were cats, but are actually dogs
incorrect_cats = np.where((preds==0) & (preds!=val_labels[:,1]))[0]
most_incorrect_cats = np.argsort(probs[incorrect_cats])[::-1][:n_view]
plots_idx(incorrect_cats[most_incorrect_cats], probs[incorrect_cats][most_incorrect_cats])

In [ ]:
#3. The images we were most confident were dogs, but are actually cats
incorrect_dogs = np.where((preds==1) & (preds!=val_labels[:,1]))[0]
most_incorrect_dogs = np.argsort(probs[incorrect_dogs])[:n_view]
plots_idx(incorrect_dogs[most_incorrect_dogs], 1-probs[incorrect_dogs][most_incorrect_dogs])

In [ ]:
#5. The most uncertain labels (ie those with probability closest to 0.5).
most_uncertain = np.argsort(np.abs(probs-0.5))
plots_idx(most_uncertain[:n_view], probs[most_uncertain])

What seems to be the case here, is that since I have made a label which often contain two of the given objects in one frame, and tought it that both are valid guesses, it's super-insecure about what to predict for them. 

Should this be ruled a success and labeled as NSFW zone? Or should we relabel data to be more clear? 

I think this will resolve itself when the network changes to account for locations and looks at the whole 1920x1080 image.

In [ ]:
val_classes

In [ ]:
from sklearn.metrics import confusion_matrix
from utils import plot_confusion_matrix
cm = confusion_matrix(val_classes, preds)
plot_confusion_matrix(cm, {'NSWF_1':0, 'NSWF_1':1, 'NSWF_1':2, 'NSWF_1':3, 'NSWF_1':4})

How train something that recognizes multiple classes per image? throw in multiple boxes and "True" per image?

Or do we train something which recognizes the classes of interest, and then simply crops out the heatmap?

Our model now currently is very confused about obscured_elements. Probably because they often overlap.